Data is hard to visualize in Tableau given its organization.

Will try to do a few things:
1. Combine multiple lobbies into one long form
2. Combine multiple rounds into one round
3. Potentially combine multiple competitions into one

For now, we will start going step by step and then figuring out how to do the others.

# Preseason Qual Clean-up

We will clean up the preseason qualifiers

Right now we have the following file structure:

Region/Preseason_Qualifier_#/Round

What we want to do is iteratively combine each folder of files starting with the round files.
Then combine the rounds by each Preseason_Qualifier_# and then hopefully combine all those by region.

We will start with the APAC_North region and then move our way down alphabetically.

## Data Structure

First we have to understand how the data even looks when we load it into Python

In [1]:
# These are the necessary packages so far
import os
import re
import pandas as pd

In [63]:
# Start with the main directory
directory = '../Outputs/APAC_North/Preseason_Qualifier_1/Round_1'

df = pd.read_csv(f"{directory}/Lobby 1.csv")
df.head()

,Unnamed: 0,Standings,Standings.1,Standings.2,Standings.3
0,NaN,Team,Team.1,Total,Round 1
1,0.0,1.0,FC Destroy,112,112
2,1.0,2.0,Dps,38,38
3,2.0,3.0,CJAM,32,32
4,3.0,4.0,sspZ,31,31


Few things to note:
1. first column seems to be a repeat of standings. we can drop that.
2. total and round 1 are duplicate we can drop that
3. we can get rid of the current column header and replace it with the existing row 1

Lets start by cleaning that up

In [64]:
# drop unnecessary columns

df = df.drop(df.columns[[0,-1]],axis=1)

In [65]:
df.head()

,Standings,Standings.1,Standings.2
0,Team,Team.1,Total
1,1.0,FC Destroy,112
2,2.0,Dps,38
3,3.0,CJAM,32
4,4.0,sspZ,31


In [66]:
df.iloc[0,1:]

Standings.1    Team.1
Standings.2     Total
Name: 0, dtype: object

In [67]:
# Rename last 2 columns columns to be our first row

df = df.rename(columns = df.iloc[0, 1:]).loc[1:]

In [68]:
# Now it's looking better.

df.head()

,Standings,Team.1,Total
1,1.0,FC Destroy,112
2,2.0,Dps,38
3,3.0,CJAM,32
4,4.0,sspZ,31
5,5.0,Joel Mark,24


In [21]:
# Lets save to a new folder
if not os.path.exists(f'{directory}/Cleaned'): # check if the folder exists, otherwise make it
    os.mkdir(f'{directory}/Cleaned')

In [69]:
df.to_csv(f'{directory}/Cleaned/Round_1_cleaned.csv', index = False)

In [37]:
# Time to repeat for all files in our folder
for file in os.listdir(directory):
    if file != "Cleaned":
        file_name = str({file}).strip("'{}").replace('.csv','') #Convert filename to string and remove excess text
        df = pd.read_csv(f"{directory}/{file}") #Open CSV as dataframe
        df = df.drop(df.columns[[0,-1]],axis=1) #Remove unneeded columns
        df = df.rename(columns = df.iloc[0, 1:]).loc[1:] #Rename headers
        df.to_csv(f'{directory}/Cleaned/{file_name}_cleaned.csv', index = False) #Save to csv without index

That was good for the round stages but our quarter, semi, and finals are going to be a bit different

In [47]:
# Make sure our directory is good and load a file
directory = '../Outputs/APAC_North/Preseason_Qualifier_1/Quarterfinals'

df = pd.read_csv(f"{directory}/Lobby 1.csv")
df.head()

,Unnamed: 0,StandingsPlacements and killsPoints,StandingsPlacements and killsPoints.1,StandingsPlacements and killsPoints.2,StandingsPlacements and killsPoints.3,StandingsPlacements and killsPoints.4,StandingsPlacements and killsPoints.5,StandingsPlacements and killsPoints.6,StandingsPlacements and killsPoints.7,StandingsPlacements and killsPoints.8,StandingsPlacements and killsPoints.9,StandingsPlacements and killsPoints.10
0,NaN,Unnamed: 0_level_1,Team,Total,"Round 1 Round 1 September 11, 2021 - 14:20 JST...","Round 1 Round 1 September 11, 2021 - 14:20 JST...","Round 2 Round 2 September 11, 2021 - 14:55 JST...","Round 2 Round 2 September 11, 2021 - 14:55 JST...","Round 3 Round 3 September 11, 2021 - 15:30 JST...","Round 3 Round 3 September 11, 2021 - 15:30 JST...","Round 4 Round 4 September 11, 2021 - 16:05 JST...","Round 4 Round 4 September 11, 2021 - 16:05 JST..."
1,NaN,Unnamed: 0_level_2,Team,Total,P,K,P,K,P,K,P,K
2,0.0,1.0,GW,77,1,16,9,12,13,5,1,17
3,1.0,2.0,FC Destroy,43,18,3,13,9,6,6,3,14
4,2.0,3.0,NOVA,39,12,2,12,0,1,12,2,2


Things to do here:
1. Get rid of first column
2. Rename first 'StandingsPlacementsand killsPoints' column to just 'Standing'
3. Change remaining column names to 2nd row from Team -> end

In [48]:
df = df.drop(df.columns[[0]],axis=1)
df.head()

,StandingsPlacements and killsPoints,StandingsPlacements and killsPoints.1,StandingsPlacements and killsPoints.2,StandingsPlacements and killsPoints.3,StandingsPlacements and killsPoints.4,StandingsPlacements and killsPoints.5,StandingsPlacements and killsPoints.6,StandingsPlacements and killsPoints.7,StandingsPlacements and killsPoints.8,StandingsPlacements and killsPoints.9,StandingsPlacements and killsPoints.10
0,Unnamed: 0_level_1,Team,Total,"Round 1 Round 1 September 11, 2021 - 14:20 JST...","Round 1 Round 1 September 11, 2021 - 14:20 JST...","Round 2 Round 2 September 11, 2021 - 14:55 JST...","Round 2 Round 2 September 11, 2021 - 14:55 JST...","Round 3 Round 3 September 11, 2021 - 15:30 JST...","Round 3 Round 3 September 11, 2021 - 15:30 JST...","Round 4 Round 4 September 11, 2021 - 16:05 JST...","Round 4 Round 4 September 11, 2021 - 16:05 JST..."
1,Unnamed: 0_level_2,Team,Total,P,K,P,K,P,K,P,K
2,1.0,GW,77,1,16,9,12,13,5,1,17
3,2.0,FC Destroy,43,18,3,13,9,6,6,3,14
4,3.0,NOVA,39,12,2,12,0,1,12,2,2


In [52]:
df = df.rename(columns = {'StandingsPlacements and killsPoints':'Standing'})
df.head()

,Standing,StandingsPlacements and killsPoints.1,StandingsPlacements and killsPoints.2,StandingsPlacements and killsPoints.3,StandingsPlacements and killsPoints.4,StandingsPlacements and killsPoints.5,StandingsPlacements and killsPoints.6,StandingsPlacements and killsPoints.7,StandingsPlacements and killsPoints.8,StandingsPlacements and killsPoints.9,StandingsPlacements and killsPoints.10
0,Unnamed: 0_level_1,Team,Total,"Round 1 Round 1 September 11, 2021 - 14:20 JST...","Round 1 Round 1 September 11, 2021 - 14:20 JST...","Round 2 Round 2 September 11, 2021 - 14:55 JST...","Round 2 Round 2 September 11, 2021 - 14:55 JST...","Round 3 Round 3 September 11, 2021 - 15:30 JST...","Round 3 Round 3 September 11, 2021 - 15:30 JST...","Round 4 Round 4 September 11, 2021 - 16:05 JST...","Round 4 Round 4 September 11, 2021 - 16:05 JST..."
1,Unnamed: 0_level_2,Team,Total,P,K,P,K,P,K,P,K
2,1.0,GW,77,1,16,9,12,13,5,1,17
3,2.0,FC Destroy,43,18,3,13,9,6,6,3,14
4,3.0,NOVA,39,12,2,12,0,1,12,2,2


In [53]:
df = df.rename(columns = df.iloc[1, 1:]).loc[2:]
df.head()

,Standing,Team,Total,P,K,P,K,P,K,P,K
2,1.0,GW,77,1,16,9,12,13,5,1,17
3,2.0,FC Destroy,43,18,3,13,9,6,6,3,14
4,3.0,NOVA,39,12,2,12,0,1,12,2,2
5,4.0,LapiLegends,37,11,1,1,11,3,4,14,0
6,5.0,Japan LFT desu,32,6,1,2,3,4,3,4,3


Now we can do the same for our other files in our other folders using a similar process.

In [54]:
# Convert above into a function
def folder_gen(directory):
    if not os.path.exists(f'{directory}/Cleaned'): # check if the folder exists, otherwise make it
        os.mkdir(f'{directory}/Cleaned')

def psq_round_cleaner(directory):
    for file in os.listdir(directory):
        if file != "Cleaned":
            file_name = str({file}).strip("'{}").replace('.csv','') #Convert filename to string and remove excess text
            df = pd.read_csv(f"{directory}/{file}") #Open CSV as dataframe
            df = df.drop(df.columns[[0,-1]],axis=1) #Remove unneeded columns
            df = df.rename(columns = df.iloc[0, 1:]).loc[1:] #Rename headers
            df.to_csv(f'{directory}/Cleaned/{file_name}_cleaned.csv', index = False) #Save to csv without index
            
def psq_quarters_up(directory):
    for file in os.listdir(directory):
        if file != "Cleaned":
            file_name = str({file}).strip("'{}").replace('.csv','') #Convert filename to string and remove excess text
            df = pd.read_csv(f"{directory}/{file}") #Open CSV as dataframe
            df = df.drop(df.columns[[0]],axis=1) #Remove unneeded columns
            df = df.rename(columns = {'StandingsPlacements and killsPoints':'Standing'}) #Rename standings column
            df = df.rename(columns = df.iloc[1, 1:]).loc[2:] #Rename headers
            df.to_csv(f'{directory}/Cleaned/{file_name}_cleaned.csv', index = False) #Save to csv without index

In [40]:
# Lets try to do this for each folder in our Preseason qualifier 1

apac_n_psq1_dir = '../Outputs/APAC_North/Preseason_Qualifier_1'

os.listdir(apac_n_psq1_dir)

['Finals', 'Quarterfinals', 'Round_1', 'Round_2', 'Round_3', 'Semifinals']

In [70]:
# Iterate through this
quarters_up = ['Quarterfinals','Semifinals','Finals']

for folder in os.listdir(apac_n_psq1_dir):
    directory = f'{apac_n_psq1_dir}/{folder}'
    folder_gen(directory)
    if folder in quarters_up:
        psq_quarters_up(directory)
    else:
        psq_round_cleaner(directory)

Now that we can clean for *one* Preseason Qualifier, time to do it for *other* Preseason QUalifiers in APAC_North!

In [61]:
pqls = ['Preseason_Qualifier_1', 'Preseason_Qualifier_2', 'Preseason_Qualifier_3', 'Preseason_Qualifier_4']

for pql in pqls:
    outter_dir = f'../Outputs/APAC_North/{pql}'
    
    if pql == ['Preseason_Qualifier_1']:
        for folder in os.listdir(outter_dir):
            directory = f'{outter_dir}/{folder}'
            folder_gen(directory)
            if folder in quarters_up:
                psq_quarters_up(directory)
            else:
                psq_round_cleaner(directory)
    else:
        for folder in os.listdir(outter_dir):
            directory = f'{outter_dir}/{folder}'
            folder_gen(directory)
            psq_quarters_up(directory)

Looks like that worked so lets do it for other regions!

In [72]:
pqls = ['Preseason_Qualifier_1', 'Preseason_Qualifier_2', 'Preseason_Qualifier_3', 'Preseason_Qualifier_4']
regions = ["APAC_North", "APAC_South", "EMEA", "North_America", "South_America"]

for region in regions:
    for pql in pqls:
        outter_dir = f'../Outputs/{region}/{pql}'

        if pql == 'Preseason_Qualifier_1':
            for folder in os.listdir(outter_dir):
                directory = f'{outter_dir}/{folder}'
                folder_gen(directory)
                if folder in quarters_up:
                    psq_quarters_up(directory)
                else:
                    psq_round_cleaner(directory)
        else:
            for folder in os.listdir(outter_dir):
                directory = f'{outter_dir}/{folder}'
                folder_gen(directory)
                psq_quarters_up(directory)

Now that we've done the Preseason Qualifiers, lets work on Challenger Circuit!

These have a similar format so I will just be running the code for Challenger Circuit

In [74]:
regions = ["APAC_North", "APAC_South", "EMEA", "North_America", "South_America"]
split = ['Split_1','Split_2']
chall_circuit = ['Challenger_Circuit_1', 'Challenger_Circuit_2', 'Challenger_Circuit_3', 'Challenger_Circuit_4']

for region in regions:
    for s in split:
        for circuit in chall_circuit:
            outter_dir = f'../Outputs/{region}/{s}/{circuit}'
            try:
                for folder in os.listdir(outter_dir):
                    directory = f'{outter_dir}/{folder}'
                    folder_gen(directory)
                    psq_quarters_up(directory)
            except FileNotFoundError:  # Not every region has the exact same rounds/splits so we move on
                continue

Now work on Split 1 specifically the playoffs and pro league. They have the same format as overall table structure so we just need to change the directories as needed.

In [75]:
regions = ["APAC_North", "APAC_South", "EMEA", "North_America", "South_America"]
split = ['Split_1','Split_2']
league = ['Playoffs', 'Pro_League']

for region in regions:
    for s in split:
        for l in league:
            outter_dir = f'../Outputs/{region}/{s}/{l}'
            try:
                for folder in os.listdir(outter_dir):
                    directory = f'{outter_dir}/{folder}'
                    folder_gen(directory)
                    psq_quarters_up(directory)
            except FileNotFoundError:  # Not every region has the exact same rounds/splits so we move on
                continue

PermissionError: [Errno 13] Permission denied: '../Outputs/APAC_North/Split_2/Pro_League/Qualifiers/Finals'